In [2]:
import numpy as np
from scipy.optimize import minimize
from scipy.linalg import expm

class QAOAPurePython:
    """
    Implementación de QAOA en Python puro usando simulación de estados cuánticos
    """

    def __init__(self, edges):
        self.edges = edges
        self.vertices = self._get_vertices()
        self.n_qubits = len(self.vertices)
        self.dim = 2 ** self.n_qubits

    def _get_vertices(self):
        """Extraer vértices únicos"""
        vertices = set()
        for edge in self.edges:
            vertices.add(edge[0])
            vertices.add(edge[1])
        return sorted(list(vertices))

    def pauli_z(self, qubit):
        """Matriz de Pauli Z en el qubit especificado"""
        I = np.eye(2)
        Z = np.array([[1, 0], [0, -1]])

        matrices = []
        for i in range(self.n_qubits):
            if i == qubit:
                matrices.append(Z)
            else:
                matrices.append(I)

        result = matrices[0]
        for mat in matrices[1:]:
            result = np.kron(result, mat)

        return result

    def pauli_x(self, qubit):
        """Matriz de Pauli X en el qubit especificado"""
        I = np.eye(2)
        X = np.array([[0, 1], [1, 0]])

        matrices = []
        for i in range(self.n_qubits):
            if i == qubit:
                matrices.append(X)
            else:
                matrices.append(I)

        result = matrices[0]
        for mat in matrices[1:]:
            result = np.kron(result, mat)

        return result

    def cost_hamiltonian_matrix(self):
        """
        Construir matriz del Hamiltoniano de costo
        H_C = sum_i (1 - Z_i)/2 + penalty * sum_{(i,j)} (1 + Z_i)(1 + Z_j)/4
        """
        H = np.zeros((self.dim, self.dim))
        I = np.eye(self.dim)

        # Término 1: suma de (1 - Z_i)/2
        for v in self.vertices:
            H += 0.5 * I - 0.5 * self.pauli_z(v)

        # Término 2: penalización por aristas no cubiertas
        penalty = 3
        for edge in self.edges:
            i, j = edge[0], edge[1]
            # (1 + Z_i)(1 + Z_j)/4
            term = I + self.pauli_z(i) + self.pauli_z(j) + self.pauli_z(i) @ self.pauli_z(j)
            H += (penalty / 4) * term

        return H

    def mixer_hamiltonian_matrix(self):
        """
        Construir matriz del Hamiltoniano mezclador
        H_M = sum_i X_i
        """
        H = np.zeros((self.dim, self.dim))

        for v in self.vertices:
            H += self.pauli_x(v)

        return H

    def initial_state(self):
        """Estado inicial: superposición uniforme |+>^n"""
        state = np.ones(self.dim) / np.sqrt(self.dim)
        return state

    def qaoa_state(self, params):
        """
        Aplicar circuito QAOA y retornar el estado final
        params: array de forma (p, 2) donde p es el número de capas
        """
        state = self.initial_state()

        H_C = self.cost_hamiltonian_matrix()
        H_M = self.mixer_hamiltonian_matrix()

        p_layers = len(params)

        for layer in range(p_layers):
            gamma = params[layer][0]
            beta = params[layer][1]

            # Aplicar exp(-i * gamma * H_C)
            U_C = expm(-1j * gamma * H_C)
            state = U_C @ state

            # Aplicar exp(-i * beta * H_M)
            U_M = expm(-1j * beta * H_M)
            state = U_M @ state

        return state

    def expectation_value(self, params):
        """Calcular <psi|H_C|psi>"""
        state = self.qaoa_state(params)
        H_C = self.cost_hamiltonian_matrix()

        expectation = np.real(np.conj(state) @ H_C @ state)
        return expectation

    def optimize(self, p_layers=2, maxiter=200):
        """Optimizar parámetros QAOA"""
        # Inicializar parámetros aleatoriamente
        np.random.seed(42)
        initial_params = np.random.uniform(0, 2*np.pi, size=(p_layers, 2))

        # Función objetivo (queremos minimizar)
        def objective(params_flat):
            params = params_flat.reshape((p_layers, 2))
            return self.expectation_value(params)

        # Optimizar
        result = minimize(
            objective,
            initial_params.flatten(),
            method='COBYLA',
            options={'maxiter': maxiter}
        )

        optimal_params = result.x.reshape((p_layers, 2))
        return optimal_params

    def get_probabilities(self, params):
        """Obtener probabilidades de medición"""
        state = self.qaoa_state(params)
        probs = np.abs(state) ** 2
        return probs

    def approximation_ratio(self, params):
        """Calcular el ratio de aproximación"""
        # Energía de QAOA
        qaoa_energy = self.expectation_value(params)

        # Energía exacta (mínimo autovalor)
        H_C = self.cost_hamiltonian_matrix()
        eigenvalues = np.linalg.eigvalsh(H_C)
        true_min = np.min(eigenvalues)

        ratio = qaoa_energy / true_min
        return ratio

    def verify_ground_state(self, params):
        """Verificar si encontramos el estado fundamental"""
        probs = self.get_probabilities(params)
        index = np.argmax(probs)

        # Vector del estado más probable
        vector = np.zeros(self.dim)
        vector[index] = 1

        # Calcular energía de ese estado
        H_C = self.cost_hamiltonian_matrix()
        energy = np.real(vector @ H_C @ vector)

        # Energía del estado fundamental
        eigenvalues = np.linalg.eigvalsh(H_C)
        ground_energy = np.min(eigenvalues)

        return np.isclose(energy, ground_energy)


# Ejemplo de uso
def solve_minimum_vertex_cover(edges):
    """Resolver el problema de Minimum Vertex Cover"""

    qaoa = QAOAPurePython(edges)

    print(f"Grafó con {qaoa.n_qubits} vértices")
    print(f"Aristas: {edges}")

    # Optimizar con 2 capas QAOA
    print("\nOptimizando parámetros QAOA...")
    optimal_params = qaoa.optimize(p_layers=2, maxiter=200)

    # Calcular ratio de aproximación
    ratio = qaoa.approximation_ratio(optimal_params)
    print(f"\nRatio de aproximación: {ratio:.4f}")

    # Verificar si encontramos el estado fundamental
    found_ground = qaoa.verify_ground_state(optimal_params)
    print(f"¿Estado fundamental encontrado?: {found_ground}")

    # Mostrar probabilidades más altas
    probs = qaoa.get_probabilities(optimal_params)
    top_indices = np.argsort(probs)[-3:][::-1]

    print("\nEstados más probables:")
    for idx in top_indices:
        binary = format(idx, f'0{qaoa.n_qubits}b')
        print(f"  |{binary}> : probabilidad = {probs[idx]:.4f}")

    return ratio, optimal_params


# Casos de prueba
if __name__ == "__main__":
    test_cases = [
        [[0, 1], [1, 2], [0, 2], [2, 3]],
        [[0, 1], [1, 2], [2, 3], [3, 0]],
        [[0, 1], [0, 2], [1, 2], [1, 3], [2, 4], [3, 4]]
    ]

    for i, edges in enumerate(test_cases):
        print(f"\n{'='*60}")
        print(f"CASO DE PRUEBA {i+1}")
        print(f"{'='*60}")
        ratio, params = solve_minimum_vertex_cover(edges)
        print(f"\nResultado final: Ratio = {ratio:.4f}")


CASO DE PRUEBA 1
Grafó con 4 vértices
Aristas: [[0, 1], [1, 2], [0, 2], [2, 3]]

Optimizando parámetros QAOA...

Ratio de aproximación: 1.4213
¿Estado fundamental encontrado?: False

Estados más probables:
  |1101> : probabilidad = 0.3617
  |1011> : probabilidad = 0.1525
  |0111> : probabilidad = 0.1525

Resultado final: Ratio = 1.4213

CASO DE PRUEBA 2
Grafó con 4 vértices
Aristas: [[0, 1], [1, 2], [2, 3], [3, 0]]

Optimizando parámetros QAOA...

Ratio de aproximación: 1.0000
¿Estado fundamental encontrado?: True

Estados más probables:
  |1010> : probabilidad = 0.5000
  |0101> : probabilidad = 0.5000
  |0111> : probabilidad = 0.0000

Resultado final: Ratio = 1.0000

CASO DE PRUEBA 3
Grafó con 5 vértices
Aristas: [[0, 1], [0, 2], [1, 2], [1, 3], [2, 4], [3, 4]]

Optimizando parámetros QAOA...

Ratio de aproximación: 1.3689
¿Estado fundamental encontrado?: False

Estados más probables:
  |10111> : probabilidad = 0.1726
  |11011> : probabilidad = 0.1726
  |10110> : probabilidad = 0.117